### LLM Chain 만들기


## 1. 환경 구성

### 1) 라이브러리 설치

In [ ]:
%pip install -q langchain langchain-openai dotenv

Note: you may need to restart the kernel to use updated packages.


### 2) OpenAI 인증키 설정
https://openai.com/

In [2]:
from dotenv import load_dotenv
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

True

In [3]:
import langchain
print(langchain.__version__)

0.3.21


## 2. LLM Chain

### 1) Prompt + LLM


In [4]:
from langchain_openai import ChatOpenAI

# model
llm = ChatOpenAI(model="gpt-3.5-turbo")

# chain 실행
result = llm.invoke("인공지능 모델의 학습 원리에 대하여 쉽게 설명해 주세요.")
print(type(result))

<class 'langchain_core.messages.ai.AIMessage'>


In [6]:
result

AIMessage(content='인공지능 모델의 학습 원리는 데이터를 입력으로 받아 패턴을 학습하여 일련의 규칙을 파악하는 과정입니다. \n\n먼저, 모델은 초기에 가중치와 편향을 랜덤한 값으로 설정한 후 입력 데이터를 받아 예측을 수행합니다. 이 예측 결과와 정답(레이블)을 비교하여 오차를 계산하고, 이 오차를 최소화하는 방향으로 가중치와 편향을 조정해나갑니다.\n\n이러한 과정을 반복하여 모델은 데이터의 패턴을 스스로 학습하게 되고, 최종적으로는 주어진 입력에 대해 정확한 결과를 예측할 수 있게 됩니다. 이렇게 학습된 모델은 새로운 데이터에 대해서도 잘 동작할 수 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 260, 'prompt_tokens': 34, 'total_tokens': 294, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BCoOafU84POdvpb5VAVhXbytnIFAs', 'finish_reason': 'stop', 'logprobs': None}, id='run-f8f0a00b-f635-4b83-94a9-875abc23a285-0', usage_metadata={'input_tokens': 34, 'output_tokens': 260, 'total_tokens': 294, 'input_token_details': {'audio': 0, 'cac

In [7]:
print(result.content)

인공지능 모델의 학습 원리는 데이터를 입력으로 받아 패턴을 학습하여 일련의 규칙을 파악하는 과정입니다. 

먼저, 모델은 초기에 가중치와 편향을 랜덤한 값으로 설정한 후 입력 데이터를 받아 예측을 수행합니다. 이 예측 결과와 정답(레이블)을 비교하여 오차를 계산하고, 이 오차를 최소화하는 방향으로 가중치와 편향을 조정해나갑니다.

이러한 과정을 반복하여 모델은 데이터의 패턴을 스스로 학습하게 되고, 최종적으로는 주어진 입력에 대해 정확한 결과를 예측할 수 있게 됩니다. 이렇게 학습된 모델은 새로운 데이터에 대해서도 잘 동작할 수 있습니다.


### 2) PromptTemplate + LLM

In [8]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("You are an expert in AI Expert. Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.")
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in AI Expert. Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.')

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

# chain 연결 (LCEL)
chain = prompt | llm
print(type(chain))

# chain 호출
result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
print(type(result))

<class 'langchain_core.runnables.base.RunnableSequence'>
<class 'langchain_core.messages.ai.AIMessage'>


In [10]:
print(result.content)

인공지능 모델의 학습 원리는 데이터를 입력으로 받아들여 이를 처리하고 패턴을 인식하는 과정입니다. 모델은 데이터를 특정한 방식으로 가공하여 학습을 통해 최적의 결정을 내리게 됩니다. 이는 입력 데이터와 기대 출력값의 차이를 최소화하는 방향으로 모델을 조정하면서 이루어집니다. 주요한 요소는 데이터의 품질과 양, 모델의 구조와 학습 알고리즘 등이 있습니다. 결국, 인공지능 모델의 학습은 데이터를 기반으로 지속적인 조정과 업데이트를 통해 정확도와 성능을 향상시키는 과정입니다.


### 3) PromptTemplate + LLM(invoke()) + StrOutputParser

In [11]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 1. 컴포넌트 정의
prompt = PromptTemplate.from_template("You are an expert in AI Expert. Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.")

llm = ChatOpenAI(model="gpt-3.5-turbo")

output_parser = StrOutputParser()

# 2. chain 생성 (LCEL)
chain = prompt | llm | output_parser
print(type(chain))

# 3. chain의 invoke 호출
result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
print(type(result))

<class 'langchain_core.runnables.base.RunnableSequence'>
<class 'str'>


In [12]:
print(result)

인공지능 모델의 학습 원리는 데이터를 사용하여 패턴을 학습하는 과정입니다. 일반적으로 입력 데이터와 원하는 출력 사이의 관계를 모델이 학습하고, 학습된 모델은 이를 기반으로 새로운 입력에 대해 정확한 출력을 예측할 수 있습니다. 이를 위해 모델은 데이터를 반복해서 입력받고, 예측 결과를 출력하여 데이터와의 차이를 최소화하는 방향으로 가중치를 조정하며 학습을 진행합니다. 이 과정을 통해 모델은 데이터 간의 복잡한 관계를 파악하고, 예측 성능을 향상시키는 것이 핵심입니다.


### 4) PromptTemplate + LLM(stream()) + StrOutputParser

In [15]:

from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 1. 컴포넌트 정의
prompt = PromptTemplate.from_template("You are an expert in AI Expert. Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.")

llm = ChatOpenAI(model="gpt-3.5-turbo")

# chain 연결 (LCEL)
chain = prompt | llm | StrOutputParser()

# 스트리밍 출력을 위한 요청
answer = chain.stream({"input": "인공지능 모델의 학습 원리"})
# 스트리밍 출력
print(answer)

for token in answer:
    # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
    print(token, end="", flush=True)

<generator object RunnableSequence.stream at 0x0000024A53000C70>
인공지능 모델의 학습 원리는 데이터를 기반으로 하는 것입니다. 모델은 입력된 데이터를 분석하여 패턴이나 규칙을 학습하고 이를 바탕으로 새로운 데이터를 예측하거나 분류할 수 있습니다. 학습 과정에서 모델은 오차를 최소화하는 방향으로 계속해서 업데이트되어 정확도를 높여나갑니다. 이러한 학습 원리에 따라 인공지능 모델은 점차 더 정확하고 효율적으로 작동하게 됩니다.

##### 2) Multiple Chains
* Multi Chain을 활용한 영화 추천 및 줄거리 요약

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 추전한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 줄거리를 3문장으로 요약해 주세요.")

# OpenAI 모델 사용
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
movie = chain1.invoke({"genre": "Drama"})  # 영화 제목 얻기
print(" 추천된 영화:", movie)  # movie 값 출력

🔹 추천된 영화: 한 편의 드라마 장르 영화로 추천드릴 만한 작품은 "그린 북"입니다. 이 영화는 흑인 피아니스트와 백인 운전사가 함께 미국 남부를 여행하며 서로에 대한 편견을 깨고 우정을 형성해나가는 이야기를 그린 작품으로, 감동적인 스토리와 훌륭한 연기로 많은 관객들의 사랑을 받았습니다. 정서적으로 감동적이고 사회적인 메시지를 담고 있는 이 영화는 드라마 장르를 좋아하는 분들에게 적극 추천드립니다.


In [17]:
# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
chain2 = (
    {"movie": chain1}  # chain1의 출력을 movie 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)
print(chain2)

# 실행: "SF" 장르의 영화 추천 및 줄거리 요약
response = chain2.invoke({"genre": "Drama"})
print("\n🔹 영화 줄거리 요약:\n", response) 

first={
  movie: ChatPromptTemplate(input_variables=['genre'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['genre'], input_types={}, partial_variables={}, template='{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.'), additional_kwargs={})])
         | ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000024A5303FEF0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000024A530D9460>, root_client=<openai.OpenAI object at 0x0000024A4FEB6AB0>, root_async_client=<openai.AsyncOpenAI object at 0x0000024A530DACC0>, model_name='gpt-3.5-turbo-0125', model_kwargs={}, openai_api_key=SecretStr('**********'))
         | StrOutputParser()
} middle=[ChatPromptTemplate(input_variables=['movie'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['movie'], input_types={}, partial_variables={}, template='{m

##### chain1과 chain2에서 영화 제목이 일관되게 전달 되도록 변경

In [18]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
import re

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
prompt1 = ChatPromptTemplate.from_template("{genre} 장르에서 추천할 만한 영화를 한 편 알려주세요.")

# Step 2: 추천된 영화의 줄거리를 요약
prompt2 = ChatPromptTemplate.from_template("{movie} 추천한 영화의 제목을 먼저 알려주시고, 줄을 바꾸어서 영화의 줄거리를 3문장으로 요약해 주세요.")

# OpenAI 모델 사용
llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

# 체인 1: 영화 추천 (입력: 장르 → 출력: 영화 제목)
chain1 = prompt1 | llm | StrOutputParser()

# Step 1: 사용자가 입력한 장르에 따라 영화 추천
movie_response = chain1.invoke({"genre": "Drama"})  # 영화 제목 얻기
print(" 추천된 영화 (원본 응답):", movie_response)

#  영화 제목만 추출 (첫 번째 큰따옴표 안의 문자열 찾기)
match = re.search(r'"(.*?)"', movie_response)
if match:
    movie_title = match.group(1)
else:
    movie_title = movie_response.split(" ")[0]  # 대안으로 첫 번째 단어 사용

print(" 추출된 영화 제목:", movie_title)

# 체인 2: 줄거리 요약 (입력: 영화 제목 → 출력: 줄거리)
chain2 = prompt2 | llm | StrOutputParser()

# 실행: chain1의 출력(영화 제목)을 chain2에 명확히 전달
response = chain2.invoke({"movie": movie_title})
print("\n 영화 줄거리 요약:\n", response)


 추천된 영화 (원본 응답): "미드나잇 인 파리"라는 영화를 추천드립니다. 이 영화는 로맨스와 판타지 요소가 가미된 로판드 장르의 작품으로, 파리의 아름다운 거리에서 벌어지는 사랑 이야기를 그린 작품입니다. 감각적인 영상미와 감동적인 이야기가 눈과 귀를 사로잡는 영화로, 여러 감정을 경험하고 싶을 때 추천하는 작품입니다.
 추출된 영화 제목: 미드나잇 인 파리

 영화 줄거리 요약:
 영화 제목: "미드나잇 인 파리" 

줄거리 요약:
미드나잇 인 파리는 누런 빛이 운 도시에 갈꾸맛을 따라 예술가인 길이 감봉을 하는 실생 기나 로맨스 보코 따색 켄 영화이다. 감봉에게 열을 달한 길이 어늉져 어늉점을 이어감하며 둘은 사 무 드러운 갈꾸맙 이엉 크게 달라지 며 빈즌의 상민향 지면서 둛의 사랑은 어늉장/나 더 저질 발전해 간다. 이 설화의 주복은 '영 더 저하이 데 데이 파트' 이다.
